In [189]:
from platform import python_version
python_version()

'3.10.6'

In [211]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

assessments = pd.read_csv('../data/assessments.csv')
studentInfo = pd.read_csv('../data/studentInfo.csv')

In [212]:
query_params = {"code_module" : "FFF",
                "code_presentation" : "2014J"}

In [278]:
assessments_hier = assessments.loc[assessments['code_module']==query_params['code_module']].loc[assessments['code_presentation']==query_params['code_presentation']].drop(columns=['code_module', 'code_presentation']).set_index(['id_assessment']).sort_index()
# display(assessments_hier)

all_exam_id_assessment = assessments.loc[assessments['code_presentation']=='2013J']
all_exam_id_assessment = all_exam_id_assessment.loc[all_exam_id_assessment['assessment_type']=='Exam']
display(all_exam_id_assessment)
# display(assessments)

studentAssessment_xxx = pd.read_csv('../data/studentAssessment.csv')

studentAssessment_xxx = studentAssessment_xxx.loc[studentAssessment_xxx[['id_assessment']].applymap(lambda x: x in all_exam_id_assessment['id_assessment'].values)['id_assessment'].values]
display(studentAssessment_xxx.loc[studentAssessment_xxx['id_assessment']==37424])

,code_module,code_presentation,id_assessment,assessment_type,date,weight
5,AAA,2013J,1757,Exam,NaN,100.0
35,BBB,2013J,15002,Exam,NaN,100.0
94,DDD,2013J,25354,Exam,261.0,100.0
113,EEE,2013J,30713,Exam,235.0,100.0
149,FFF,2013J,34885,Exam,236.0,100.0
185,GGG,2013J,37424,Exam,229.0,100.0


,id_assessment,id_student,date_submitted,is_banked,score


In [252]:
# studentAssessment = pd.read_csv('../data/studentAssessment.csv')
# sum(studentAssessment['id_assessment']==34911)

0

In [214]:
studentInfo_hier = studentInfo.loc[studentInfo['code_module']==query_params['code_module']].loc[studentInfo['code_presentation']==query_params['code_presentation']].drop(columns=['code_module', 'code_presentation']).set_index(['id_student']).sort_index()
# get only first try students
studentInfo_hier = studentInfo_hier.loc[studentInfo_hier.num_of_prev_attempts==0].drop(columns=['num_of_prev_attempts'])
chosen_id_students = studentInfo_hier.index.values
studentInfo_hier.head()
chosen_id_students.shape

(2034,)

In [215]:
fake_fill = pd.DataFrame(data = {'id_assessment':assessments_hier.index.values}|{col_name: [np.nan for _ in range(assessments_hier.index.values.shape[0])]
                                 for col_name in ['date_submitted','score']}
)
display(fake_fill)
display(fake_fill.assign(id_student=[35747 for _ in range(assessments_hier.shape[0])]).set_index(['id_student']))

,id_assessment,date_submitted,score
0,34899,NaN,NaN
1,34900,NaN,NaN
2,34901,NaN,NaN
3,34902,NaN,NaN
4,34903,NaN,NaN
5,34904,NaN,NaN
6,34905,NaN,NaN
7,34906,NaN,NaN
8,34907,NaN,NaN
9,34908,NaN,NaN


,id_assessment,date_submitted,score
id_student,,,
35747,34899,NaN,NaN
35747,34900,NaN,NaN
35747,34901,NaN,NaN
35747,34902,NaN,NaN
35747,34903,NaN,NaN
35747,34904,NaN,NaN
35747,34905,NaN,NaN
35747,34906,NaN,NaN
35747,34907,NaN,NaN


In [216]:
complete_empty_studentAssessment_tmp = pd.concat([fake_fill.assign(id_student=[id_student for _ in range(assessments_hier.shape[0])]).set_index(['id_student']) for id_student in chosen_id_students]).reset_index().set_index(['id_student','id_assessment'])
display(complete_empty_studentAssessment_tmp.tail())

date_submitted  score
id_student id_assessment                       
2702660    34907                     NaN    NaN
           34908                     NaN    NaN
           34909                     NaN    NaN
           34910                     NaN    NaN
           34911                     NaN    NaN

In [240]:
studentAssessment_tmp = pd.read_csv('../data/studentAssessment.csv')
# print(studentAssessment_tmp.shape)

print(sum(studentAssessment_tmp[['id_student']].applymap(lambda x: x in chosen_id_students)['id_student'].values))

studentAssessment_tmp = studentAssessment_tmp.loc[studentAssessment_tmp[['id_student']].applymap(lambda x: x in chosen_id_students)['id_student'].values]
studentAssessment_tmp = studentAssessment_tmp.loc[studentAssessment_tmp[['id_assessment']].applymap(lambda x: x in assessments_hier.index.values)['id_assessment'].values]

studentAssessment_tmp = studentAssessment_tmp.set_index(['id_student', 'id_assessment'])

print(studentAssessment_tmp.shape)
print(studentAssessment_tmp.index.get_level_values(0).unique().shape)
print(all(ids in chosen_id_students for ids in studentAssessment_tmp.index.get_level_values(0).unique()))

14872
(14193, 3)
(1573,)
True


In [243]:
for id_student in chosen_id_students:
    if id_student in studentAssessment_tmp.index.get_level_values(0).values:
        for id_assessment in studentAssessment_tmp.loc[id_student].index.values:
            complete_empty_studentAssessment_tmp.loc[(id_student, id_assessment)] = studentAssessment_tmp.loc[(id_student, id_assessment)]
# studentAssessment_tmp.index.get_level_values(1).values

In [250]:
# display(complete_empty_studentAssessment_tmp.iloc[[45,856]])
# display(complete_empty_studentAssessment_tmp.loc[172493])
studentAssessment_tmp.index.get_level_values(1).sort_values().value_counts()

34899    1563
34900    1374
34904    1270
34901    1221
34905    1201
34902    1146
34906    1108
34907    1090
34909    1077
34908    1074
34910    1052
34903    1017
Name: id_assessment, dtype: int64

In [210]:
print(studentAssessment_tmp.index.get_level_values(0).unique())
print(studentAssessment_tmp.shape)
print(chosen_id_students.shape)
# display(complete_empty_studentAssessment_tmp.get_)
print(35747 in studentAssessment_tmp.index.get_level_values(0))


Int64Index([507743, 583056, 645405, 680231, 683375, 698407, 1881231, 1962049], dtype='int64', name='id_student')
(8, 2)
(2034,)
False


In [195]:
complete_empty_studentAssessment_tmp = pd.concat([fake_fill.assign(id_student=[id_student for _ in range(assessments_hier.shape[0])]).set_index(['id_student']) for id_student in chosen_id_students]).reset_index().set_index(['id_student','id_assessment'])
display(complete_empty_studentAssessment_tmp.tail())
#         .loc[35747])
# 35747
# def get_right_studentAssessment_data(query_params = {"code_module" : "FFF", "code_presentation" : "2014J"}, chosen_id_students):
#     studentAssessment_tmp = pd.read_csv('data/source_data/studentAssessment.csv')
#     studentAssessment_tmp = studentAssessment.loc[studentAssessment[['id_assessment']].applymap(lambda x: x in assessments_hier.index.values)['id_assessment'].values].set_index(['id_student', 'id_assessment']).sort_index().reset_index().set_index(['id_student'])
#     complete_empty_studentAssessment_tmp = pd.concat([pd.DataFrame(fake_fill.reindex([id_student for _ in range(assessments_hier.shape[0])])) for id_student in studentInfo_hier.index.values])
    
studentAssessment_tmp = pd.read_csv('../data/studentAssessment.csv')
studentAssessment_tmp = studentAssessment_tmp.filter(items=chosen_id_students, axis=0)
# display(studentAssessment_tmp['id_student'] in )
studentAssessment_tmp = studentAssessment_tmp.loc[studentAssessment_tmp[['id_assessment']].applymap(lambda x: x in assessments_hier.index.values)['id_assessment'].values].set_index(['id_student,id_assessment']).sort_index().drop(columns=['is_banked'])
studentAssessment_tmp

# studentAssessment_tmp.loc[31296].combine_first

# id_student = 31296

# very_tmp = fake_fill.assign(id_student=[id_student for _ in range(assessments_hier.shape[0])]).set_index(['id_student'])
# very_tmp
# fake_fill

# very_tmp.reset_index().combine_first(studentAssessment_tmp.loc[31296].reset_index()).set_index(['id_student'])
# studentAssessment_tmp.loc[31296].reset_index()
# studentAssessment_tmp.loc[31296]

# for id_student in chosen_id_students:
#     if id_student in studentAssessment_tmp.index.values:
#         print(id_student)
#         for id_assessment in assessments_hier.index.values:
#             print('j',id_assessment)
#             if id_assessment in studentAssessment_tmp.loc[id_student].index.values:
#                 print(f'{id_student} {id_assessment}')
#                 complete_empty_studentAssessment_tmp.loc[(id_student, id_assessment)] = studentAssessment_tmp.loc[(id_student, id_assessment)]

#                 complete_empty_studentAssessment_tmp.loc[id_student].reset_index().combine_first(studentAssessment_tmp.loc[id_student].reset_index()).set_index(['id_student']).values
#     print(studentAssessment_tmp.loc[id_student])
#     = studentAssessment_tmp.loc[id_student].reset_index().combine_first(fake_fill.assign(id_student=[id_student for _ in range(assessments_hier.shape[0])]).reset_index()).set_index(['id_student'])
# studentAssessment_tmp.loc[31296]

# complete_empty_studentAssessment_tmp.head()
# display(studentAssessment_tmp.loc[(2702660, 34907)])
# display(complete_empty_studentAssessment_tmp.loc[(2702660, 34907)])

# display(studentAssessment_tmp.head())
# display(complete_empty_studentAssessment_tmp.loc[(31296,34899)])
# print(studentAssessment.reset_index().index)
# studentAssessment_tmp.head()
# print(34899 in chosen_)

date_submitted  score
id_student id_assessment                       
2702660    34907                     NaN    NaN
           34908                     NaN    NaN
           34909                     NaN    NaN
           34910                     NaN    NaN
           34911                     NaN    NaN

,id_assessment,date_submitted,score
id_student,,,
507743,34905,45,86.0
583056,34906,95,56.0
645405,34903,198,87.0
680231,34904,32,92.0
683375,34909,219,86.0
698407,34899,23,56.0
1881231,34904,58,89.0
1962049,34908,184,85.0


In [148]:
display(complete_empty_studentAssessment_tmp.head())
display(complete_empty_studentAssessment_tmp.loc[35796])
# studentAssessment = pd.read_csv('../data/assessments.csv')

,date_submitted,score
id_assessment,,
34899,NaN,NaN
34900,NaN,NaN
34901,NaN,NaN
34902,NaN,NaN
34903,NaN,NaN


KeyError: 35796

In [ ]:
studentAssess

In [26]:
studentInfo_hier.num_of_prev_attempts.value_counts()

0    2034
1     244
2      66
3      14
4       6
6       1
Name: num_of_prev_attempts, dtype: int64

In [42]:
# filtering students
# 1) first attempt - to get behaviour

In [88]:
doll1 = pd.DataFrame(data=[[2,3], [np.nan, 4]])
doll2 = pd.DataFrame(data=[[2,3], [5, np.nan], [3,3]], index = [3,1,2])

In [89]:
doll1

,0,1
0,2.0,3
1,NaN,4


In [90]:
doll2

,0,1
3,2,3.0
1,5,NaN
2,3,3.0


In [91]:
doll1.combine_first(doll2)

,0,1
0,2.0,3.0
1,5.0,4.0
2,3.0,3.0
3,2.0,3.0


In [97]:
doll1

,0,1
0,2.0,3
1,NaN,4


In [129]:
doll1.assign(ala=[4,4])

,0,1,ala
0,2.0,3,4
1,NaN,4,4
